In [61]:
import numpy as np

class Tfidf(object):
    def __init__(self, collection, stop_words=[], min_count=3 ,max_vocab=50000):
        '''collection is a list of strings'''
        self.word2count = {} # map each word to its total number of occurences in the whole collection
        self.word2count_doc = {} # map each word to the number of documents in which it appears
        self.word2ind = {} # map each word in the collection to a unique index
        self.ind2word = {}
        self.word2idf = {} # map each word to its idf
        self.collection = collection
        self.documents = [document.split() for document in collection]
        
        for document in self.documents:
            document_unique_words = set()
            for word in document:
                document_unique_words.add(word)
                if word in self.word2count:
                    self.word2count[word] += 1
                else:
                    self.word2count[word] = 1
            for word in document_unique_words:
                if word in self.word2count_doc:
                    self.word2count_doc[word] += 1
                else:
                    self.word2count_doc[word] = 1
                  
        self.word2count = {el[0]:el[1] for el in sorted(self.word2count.items(), key=lambda item: item[1],
                                                        reverse=True)} # sort by values (descending)
        self.unique_words = list(self.word2count.keys())[:max_vocab]
        self.unique_words = [word for word in self.unique_words if word not in stop_words \
                             and self.word2count[word] > min_count] # remove stopwords
        self.count_unique_words = len(self.unique_words)
        self.word2ind = dict(zip(self.unique_words,range(self.count_unique_words)))
        self.ind2word = {v:k for k,v in self.word2ind.items()}
        self.count_documents = len(collection)        
        
        # compute the idf of unqiue words in the collection
        for word in self.word2ind.keys():
            count = self.word2count_doc[word]
            idf = np.log(self.count_documents / (1 + count)) + 1 
            self.word2idf[word] = idf

    def getWordFromInd(self, ind):
        return self.ind2word[ind]

    def getListWords(self):
        return self.unique_words

    def tf(self, document):
        '''
        return the frequency of each unique word in document
        document is a list of strings
        '''
        word2frequency = {}
        for word in document:
            word2frequency[word] = word2frequency.get(word, 0) + 1 # increment, creating key if it doesn't already exist
        return word2frequency

    def transform(self, collection):
        documents = [document.split() for document in collection] # tokenize documents in the collection
        tfidf_mat = np.zeros((len(documents), self.count_unique_words)) # intialize tfidf matrix with zeros
        # compute tfidf 
        for ind, document in enumerate(documents):
            word2frequency = self.tf(document)
            for word in word2frequency.keys():
                if word in self.word2ind:
                    tfidf_mat[ind, self.word2ind[word]] = word2frequency[word] * self.word2idf[word]        
        return tfidf_mat

In [62]:
import os
data_path = 'data'

def read_data(subset):
    with open(os.path.join(data_path, '{}_documents.txt'.format(subset)), 'r') as f1, \
        open(os.path.join(data_path, '{}_labels.txt'.format(subset)), 'r') as f2:
        documents = []
        labels = []
        for line in f1:
            documents.append(line)
        f1.close()
        for line in f2:
            labels.append(int(line))
        return documents, labels
              

collection_train, labels_train = read_data('train')
collection_valid, labels_valid = read_data('valid')
collection_test, labels_test = read_data('test')

In [63]:
tfidf = Tfidf(collection_train)
X_train = tfidf.transform(collection_train)

In [64]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(multi_class='auto', solver='lbfgs', max_iter=200) 
classifier.fit(X_train, labels_train)

/data/home/moussa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [65]:
X_valid = tfidf.transform(collection_valid)

In [66]:
y_valid_predicted = classifier.predict(X_valid)

In [67]:
print(sum(y_valid_predicted == labels_valid) / len(labels_valid))

0.9406


In [82]:
l = y_valid_predicted == labels_valid

In [83]:
indices = [i for i, x in enumerate(list(l)) if x == False]

In [84]:
indices

[6,
 8,
 18,
 40,
 51,
 86,
 105,
 167,
 171,
 221,
 248,
 276,
 286,
 289,
 293,
 331,
 366,
 373,
 382,
 396,
 426,
 428,
 460,
 464,
 467,
 475,
 483,
 494,
 511,
 515,
 526,
 556,
 568,
 571,
 577,
 581,
 672,
 686,
 694,
 723,
 730,
 736,
 762,
 771,
 783,
 786,
 791,
 792,
 801,
 813,
 846,
 856,
 862,
 869,
 891,
 899,
 904,
 914,
 923,
 924,
 935,
 994,
 1050,
 1068,
 1075,
 1092,
 1124,
 1126,
 1130,
 1132,
 1167,
 1176,
 1236,
 1237,
 1257,
 1281,
 1283,
 1311,
 1325,
 1359,
 1366,
 1385,
 1395,
 1421,
 1428,
 1434,
 1457,
 1518,
 1523,
 1550,
 1572,
 1579,
 1599,
 1607,
 1612,
 1634,
 1641,
 1644,
 1651,
 1656,
 1671,
 1679,
 1693,
 1699,
 1803,
 1804,
 1810,
 1827,
 1836,
 1857,
 1866,
 1877,
 1880,
 1908,
 1925,
 1926,
 1952,
 1958,
 1969,
 2000,
 2004,
 2023,
 2055,
 2111,
 2141,
 2162,
 2177,
 2190,
 2206,
 2233,
 2238,
 2301,
 2307,
 2308,
 2318,
 2327,
 2344,
 2353,
 2373,
 2380,
 2389,
 2421,
 2478,
 2489,
 2490,
 2497,
 2520,
 2561,
 2590,
 2625,
 2643,
 2679,
 2689,

In [90]:
labels_valid[18], y_valid_predicted[18]

(3, 4)

In [92]:
collection_valid[18]

'  افادت السفارة البريطانية في بيان ، ان " الطلاب الفلسطينيين في لبنان سيتمكنون من الاستفادة من دعم جديد تقدمه المملكة المتحدة للمساهمة في مساعدة الاونروا على ابقاء مدارسها مفتوحة للعام الدراسي 2015/2016 ".  واعلن وزير التنمية الدولية البريطاني ديسموند سواين ان بلاده ستقدم دعما اضافيا بقيمة 3 ملايين جنيه استرليني للاونروا لكي يتسنى لكل طالب فلسطيني ان يذهب الى المدرسة .  واضاف البيان ان هذا الدعم يساعد " الاونروا " في تغطية العجز الذي لم تشهده المؤسسة من قبل ، وقيمته 101 مليون دولار اميركي ، وفي اعادة فتح 685 مدرسة مع بداية العام الدراسي . وتقدم " الاونروا " خدمات تعليمية اساسية لنصف مليون لاجئ فلسطيني في لبنان ، والاردن ، وسوريا ، وقطاع غزة والضفة الغربية . يذكر ان وزارة التنمية الدولية البريطانية هي ثالث اكبر ممول لميزانية الانروا العامة وميزانية الطوارئ .  وفي هذا السياق قال سواين : " تلتزم المملكة المتحدة دعم اللاجئين الفلسطينيين عبر الاونروا ، ونعتبر التعليم اولوية مهمة . وتضطلع الاونروا بدور اساسي في منطقة غير مستقرة حيث توفر الخدمات الاساسية والمساعدات الانسانية . كما توفر بيئة 